

1.   XBRLファイルを取得する
    * XBRLファイルは、EDINETから取得する
    * EDINET APIを使用：https://disclosure.edinet-fsa.go.jp/EKW0EZ0015.html
1.   取得したXBRLファイルから必要な情報を取得する
1.   






In [0]:
import requests
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json


In [0]:
# 有価証券報告書
docTypeCodes = ['120']


class Edinet:
    def __init__(self, _since, _until, _type):
        self.base_url = "https://disclosure.edinet-fsa.go.jp"
        self.since = _since
        self.until = _until
        self.type = _type

    def main(self):
        docs = self.get_documents_between(self.since, self.until, self.type)
        for doc in docs:
            print(doc)

    # 提出された書類を取得する
    def get_documents(self, _date, _type):
        count, retry = 0, 3
        url = self.base_url + "/api/v1/documents.json"
        params = {'date': _date, 'type': _type}
        while True:
            try:
                response = requests.get(url=url, params=params)
                return response.text
            except Exception:
                print(f"アクセスに失敗しました。[ {count} ]")
                if count < retry:
                    count += 1
                    time.sleep(2)
                    continue
                else:
                    raise

    def get_documents_between(self, _since, _until, _type):
        docs = []
        for _date in pd.date_range(_since, _until, freq="B", tz="Asia/Tokyo"):
            time.sleep(2)
            print(_date.strftime('%Y-%m-%d'))
            res = self.get_documents(_date.strftime('%Y-%m-%d'), _type)
            j = json.loads(res)
            d = self._select_data(j.get("results"))
            docs.append(d)
        return docs

    def _select_data(self, results):
        sdata = []
        for _data in results:
            if self._is_yuho(_data):
                tdata = {}
                tdata['seqNumber'] = _data.get("seqNumber")
                tdata["docID"] = _data.get("docID")
                tdata["filerName"] = _data.get("filerName")
                sdata.append(tdata)
        return sdata

    def _is_yuho(self, result):
        # XBRLファイルがなければ、NG
        if result.get('xbrlFlag') == '0':
            return False

        if result.get('docTypeCode') in docTypeCodes:
            return True

        return False


In [65]:
edinet = Edinet("2020-04-01", "2020-04-30", 2)
edinet.main()
# h = edinet.get_documents("2020-04-01", 2)
# d = json.loads(h)
# s = edinet._select_data(d['results'])
# json.dumps(s, ensure_ascii=False)


2020-04-01
2020-04-02
2020-04-03
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
2020-04-30
[{'seqNumber': 198, 'docID': 'S100ICUK', 'filerName': 'Ｓｐｉｂｅｒ株式会社'}, {'seqNumber': 214, 'docID': 'S100ICV6', 'filerName': 'Ｓｐｉｂｅｒ株式会社'}, {'seqNumber': 221, 'docID': 'S100ICVH', 'filerName': 'Ｓｐｉｂｅｒ株式会社'}, {'seqNumber': 227, 'docID': 'S100ICVT', 'filerName': 'Ｓｐｉｂｅｒ株式会社'}]
[]
[{'seqNumber': 10, 'docID': 'S100IC0O', 'filerName': '大和アセットマネジメント株式会社'}, {'seqNumber': 13, 'docID': 'S100I7N6', 'filerName': '三菱ＵＦＪ国際投信株式会社'}, {'seqNumber': 18, 'docID': 'S100ICOZ', 'filerName': '大和アセットマネジメント株式会社'}, {'seqNumber': 24, 'docID': 'S100ID7Y', 'filerName': 'ゴールドマン・サックス・アセット・マネジメント株式会社'}, {'seqNumber': 28, 'docID': 'S100I87X', 'filerName': '三菱ＵＦＪ国際投信株式会社'}, {'seqNumber': 34, 'docID': 'S100ICPG', 'filerName': '大和アセットマネジメント株式会社'}, {'seqNumber': 37, 'docID': 'S100I7NF',